In [1]:
import duckdb
import pandas as pd

In [2]:
# Init DB
con = duckdb.connect()

# Init Data
con.execute("""
CREATE TABLE sales (
  customer_id VARCHAR,
  order_date DATE,
  product_id INTEGER
);

INSERT INTO sales VALUES
  ('A', '2021-01-01', 1),
  ('A', '2021-01-01', 2),
  ('A', '2021-01-07', 2),
  ('A', '2021-01-10', 3),
  ('A', '2021-01-11', 3),
  ('A', '2021-01-11', 3),
  ('B', '2021-01-01', 2),
  ('B', '2021-01-02', 2),
  ('B', '2021-01-04', 1),
  ('B', '2021-01-11', 1),
  ('B', '2021-01-16', 3),
  ('B', '2021-02-01', 3),
  ('C', '2021-01-01', 3),
  ('C', '2021-01-01', 3),
  ('C', '2021-01-07', 3);

CREATE TABLE menu (
  product_id INTEGER,
  product_name VARCHAR,
  price INTEGER
);

INSERT INTO menu VALUES
  (1, 'sushi', 10),
  (2, 'curry', 15),
  (3, 'ramen', 12);

CREATE TABLE members (
  customer_id VARCHAR,
  join_date DATE
);

INSERT INTO members VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');
""")


# Question 1 - What is the total amount each customer spent at the restaurant?

## 🧠 Thought Process

### 🎯 Goal
Calculate the total amount each customer spent at the restaurant.

---

### 💼 Business Context
Understanding how much each customer spends is important for identifying high-value customers. These insights can further be used in:

- **Loyalty programs**
- **Targeted promotions**
- **Upselling strategies**

---

### 🔍 Problem Breakdown
- **Data Source**: `sales` tells us what each customer ordered; `menu` has the price.
- **Join Needed**: Yes — match `sales.product_id` with `menu.product_id`.
- **Grouping**: Group by `customer_id` to calculate total per customer.
- **Aggregation**: Sum all the product prices each customer ordered.
- **Assumption**: Each row in `sales` represents **one unit** of the product, i.e., no quantity column.

---

### 🛠 Approach & SQL Explanation
To solve this, I joined the `sales` table with the `menu` table using `product_id`, then grouped the results by customer and summed the prices.

---

### ✅ Result Validation
Manually verified for customer C:

- Three ramen ($12) → `3 × 12 = $36` ✅
- Total number of rows also matched the number of distinct customer IDs


In [3]:
query = """
SELECT 
    s.customer_id, SUM(m.price) AS total_spent
FROM
    sales AS s
        INNER JOIN
    menu AS m ON s.product_id = m.product_id
GROUP BY s.customer_id;
"""

result = con.execute(query).fetchdf()
result

,customer_id,total_spent
0,A,76.0
1,B,74.0
2,C,36.0


# Question 2 - How many days has each customer visited the restaurant?

## 🧠 Thought Process

### 🎯 Goal
Calculate the total number of **distinct days** each customer visited the restaurant.

---

### 💼 Business Context
Visit frequency is a key indicator of customer engagement and retention. Knowing how often each customer comes can help answer questions such as:

- Who are the frequent visitors that return regularly?
- Who might be loyal but not high-spending, and therefore worth nurturing?
- Is customer A visiting once a month, or every other day?

The insights gained from this analysis can inform:

- Rewarding frequent visitors
- Timing promotions (e.g., targeting customers after X days of inactivity)
- Segmenting customers into high-frequency vs. low-frequency groups for tailored strategies

---

### 🔍 Problem Breakdown
- **Data Source**: The `sales` table contains both `customer_id` and `order_date`, which are sufficient to answer this question.
- **Join Needed**: No
- **Grouping**: Group by `customer_id`
- **Aggregation**: Count the number of **unique** `order_date` values per customer

---

### 🧭 Assumptions
- Each row in `sales` represents a separate order (i.e., a unique transaction)
- A customer may place multiple orders on the same day, but that still counts as **one visit**
- All `order_date` values are valid and correctly recorded

---

### 🛠 Approach & SQL Explanation
To solve this, I used DISTINCT to isolate the unique visit dates, then applied COUNT to determine the number of visits per customer, grouping the results by customer_id.

---

### ✅ Result Validation
Manually verified for customer C:

- Customer C has 6 rows in total
- Two rows share the same `order_date`, so the number of **distinct visit days** is 4, and the SQL result matched this expected value


In [4]:
query = """
SELECT 
    customer_id, COUNT(DISTINCT CAST(order_date AS DATE)) AS visit_day_count
FROM
    sales
GROUP BY customer_id;
"""

result = con.execute(query).fetchdf()
result

,customer_id,visit_day_count
0,B,6
1,C,2
2,A,4


# Question 3 - What was the first item from the menu purchased by each customer?

## 🧠 Thought Process

### 🎯 Goal
Find the first item (or items) each customer purchased from the menu.

---

### 💼 Business Context
Understanding what customers choose during their first visit can help identify:
- The most attractive or appealing items on the menu
- Entry-level dishes that encourage repeat visits
- Product bundling opportunities for first-time visitors

This insight can be applied in new customer onboarding strategies or first-visit discount offers.

---

### 🔍 Problem Breakdown
- **Data Source**: The `sales` table provides `customer_id`, `order_date`, and `product_id`. The `menu` table provides `product_name`.
- **Join Needed**: Yes — to retrieve the product name, join `sales.product_id` with `menu.product_id`.
- **Grouping**: Not required for the final result, but `customer_id` is used in the subquery to compute the earliest order date.
- **Aggregation**: Use `MIN(order_date)` in a subquery to identify the first purchase date per customer.

---

### 🧭 Assumptions
- A customer may purchase multiple products on their first visit.
- All such products from that day should be returned.
- There are no duplicate `order_date` values with conflicting data.

---

### 🛠 Approach & SQL Explanation
To solve this, I first used `MIN(order_date)` to identify the earliest purchase date for each customer. This is done in a subquery (aliased as `first_day`), which is then joined back to the `sales` table using both `customer_id` and `order_date`. Finally, the `sales` table is joined with the `menu` table to retrieve the product names.

---

### ✅ Result Validation
Manually verified for customer A:
- On their first visit (`2021-01-01`), customer A purchased items with `product_id` = 1 and 2.
- These map to 'sushi' and 'curry' respectively — the result matched this expectation.


In [5]:
query = """
SELECT
    s.customer_id,
    s.order_date AS first_purchase_date,
    m.product_name
FROM(
    SELECT 
        customer_id,
        MIN(order_date) AS first_purchase_date
    FROM
        sales
    GROUP BY customer_id
) AS first_day
INNER JOIN sales s 
    ON first_day.customer_id = s.customer_id
    AND first_day.first_purchase_date = s.order_date
INNER JOIN menu m ON s.product_id = m.product_id;
"""

result = con.execute(query).fetchdf()
result

,customer_id,first_purchase_date,product_name
0,A,2021-01-01,sushi
1,B,2021-01-01,curry
2,C,2021-01-01,ramen
3,A,2021-01-01,curry
4,C,2021-01-01,ramen


# Question 4 - What is the most purchased item on the menu and how many times was it purchased by all customers?

## 🧠 Thought Process

### 🎯 Goal
Determine the most popular item on the menu by counting how many times each item was purchased across all customers.

---

### 💼 Business Context
Understanding item popularity can help restaurant owners:

- Identify customer favorites
- Optimize inventory and kitchen preparation
- Adjust pricing based on demand
- Feature top items in promotions or bundles

This insight is essential for **menu engineering and operational planning**.

---

### 🔍 Problem Breakdown
- **Data Source**: The `sales` table records all transactions and contains `product_id`. The `menu` table provides the corresponding `product_name`.
- **Join Needed**: Yes — to map `product_id` to `product_name`, join `sales` with `menu`.
- **Grouping**: Group by `product_id` or `product_name` to aggregate purchases per item.
- **Aggregation**: Use `COUNT(product_id)` to calculate total purchases for each item.

---

### 🧭 Assumptions
- Each row in `sales` represents a single unit of purchase (no quantity column exists).
- All product IDs in `sales` exist in the `menu` table.

---

### 🛠 Approach & SQL Explanation
To solve this, I grouped the `sales` data by `product_id` and counted how many times each product was purchased using `COUNT()`. I then joined the `menu` table to retrieve the product name.

---

### ✅ Result Validation
Manually checked:
- `ramen` appeared 8 times in the `sales` table, which matched the result in the query output.
- The item with the highest purchase count was returned correctly.

---

In [6]:
query = """
SELECT
    m.product_name,
    COUNT(s.product_id) AS amount_of_purchase
FROM sales s
INNER JOIN menu m ON s.product_id = m.product_id
GROUP BY m.product_name;
"""

result = con.execute(query).fetchdf()
result

,product_name,amount_of_purchase
0,sushi,3
1,ramen,8
2,curry,4


# Question 5 - Which item was the most popular for each customer?

## 🧠 Thought Process

### 🎯 Goal
Identify the most frequently purchased item(s) for each customer.

---

### 💼 Business Context
Understanding customer preferences is essential for personalization and targeted marketing. By identifying the most purchased item(s) per customer, the restaurant can:

- **tailor promotions**
- **suggest relevant items**
- **design loyalty perks based on individual tastes**

---

### 🔍 Problem Breakdown
- **Data Source**: The `sales` table logs each transaction and contains `product_id`, while the `menu` table maps `product_id` to `product_name`.
- **Join Needed**: Yes — join `sales.product_id` with `menu.product_id` to retrieve item names.
- **Grouping**: Group by `customer_id` and `product_id` to calculate item-wise purchase counts per customer.
- **Aggregation**: Use `COUNT(product_id)` to determine purchase frequency per item.
- **Filtering**: Select the item(s) with the highest purchase frequency for each customer.

---

### 🧭 Assumptions
- A customer may have multiple items tied for the highest purchase frequency. In such cases, all of them should be included in the final output.

---

### 🛠 Approach & SQL Explanation
1. **Step 1**: Aggregate purchase frequencies for each `(customer_id, product_id)` using `COUNT(*)`.
2. **Step 2**: Determine the maximum purchase frequency per `customer_id` using `MAX()` or a window function such as `RANK()`.
3. **Step 3**: Filter the items that match the maximum frequency per customer.
4. **Step 4**: Join the result with the `menu` table to retrieve the corresponding `product_name`.

This approach ensures that we correctly capture all most-frequently-purchased items for each customer, even when there are ties.

---

### ✅ Result Validation
Manually verified for customer A:
- Purchased product_id 3 (ramen) 3 times — the highest among all items.
- Verified that ties are also properly included when applicable.

---

### 🔗 Source
- RANK()&DENSE RANK(): https://www.geeksforgeeks.org/rank-and-dense-rank-in-sql-server/

In [7]:
# First Approach - CTE + JOIN
query = """
WITH purchase_stats AS (
    SELECT
        customer_id,
        product_id,
        COUNT(product_id) AS purchase_frequency
    FROM sales s
    GROUP BY customer_id, product_id
),
max_freq AS (
    SELECT
        customer_id,
        MAX(purchase_frequency) AS max_freq
    FROM purchase_stats
    GROUP BY customer_id
)

SELECT
    ps.customer_id,
    m.product_name,
    ps.purchase_frequency
FROM
    purchase_stats ps
INNER JOIN max_freq mf 
   ON ps.customer_id = mf.customer_id
  AND ps.purchase_frequency = mf.max_freq
INNER JOIN menu m
   ON ps.product_id = m.product_id;

"""


# Second Approach - CTE + window function
query = """
WITH ranked_purchases AS (
    SELECT
        s.customer_id,
        m.product_name,
        COUNT(*) AS purchase_freq,
        RANK() OVER(
            PARTITION BY s.customer_id
            ORDER BY COUNT(*) DESC
        ) AS rnk
    FROM sales s
    JOIN menu m
    ON s.product_id = m.product_id
    GROUP BY s.customer_id, m.product_name
)
SELECT 
    customer_id, 
    product_name, 
    purchase_freq
FROM ranked_purchases
WHERE rnk = 1;

"""

result = con.execute(query).fetchdf()
result

,customer_id,product_name,purchase_freq
0,C,ramen,3
1,B,sushi,2
2,B,ramen,2
3,B,curry,2
4,A,ramen,3


# Question 6 - Which item was purchased first by the customer after they became a member?

## 🧠 Thought Process

### 🎯 Goal

Identify the first item(s) each customer purchased after joining the membership program.

---

### 💼 Business Context

Understanding the first item purchased by new members provides insight into:

* What draws customers in right after joining
* Which products are best suited for **onboarding promotions**
* Opportunities to design **personalized welcome flows** or **starter bundles**

These early behaviors often reflect initial interests and can be critical for **retention strategies**.

---

### 🔍 Problem Breakdown

* **Data Source**: The `sales` table logs each transaction and includes `order_date` and `product_id`. The `members` table contains `customer_id` and `join_date`. The `menu` table maps `product_id` to `product_name`.
* **Join Needed**:

  * Join `sales.customer_id` with `members.customer_id` to determine whether the purchase occurred after the join date
  * Join `sales.product_id` with `menu.product_id` to retrieve item names
* **Filtering**: Only include purchases where `order_date >= join_date`
* **Window Function**: Use `RANK()` to identify the first purchase per customer after joining
* **Grouping**: Implicitly used within the window function via `PARTITION BY customer_id`
* **Aggregation**: Not needed, as we care about event order rather than totals

---

### 🧭 Assumptions

* A customer may have made multiple purchases on their first day as a member — include **all tied items**
* We assume that `order_date` is sufficient for determining purchase sequence (no need for timestamps)

---

### 🛠 Approach & SQL Explanation

1. **Step 1**: Join `sales` and `members` tables to retrieve each customer's join date alongside their order history.
2. **Step 2**: Filter to retain only those purchases that happened on or after the `join_date`.
3. **Step 3**: Use a window function `RANK()` to assign a rank to each post-membership purchase per customer based on `order_date`.
4. **Step 4**: Keep only those records where the rank equals 1 — these represent the customer’s first purchase(s) after joining.
5. **Step 5**: Join with the `menu` table to translate `product_id` into `product_name`.

This logic ensures that even if a customer bought multiple items on the same day they joined, all those tied first purchases are included.

---

### ✅ Result Validation

Manually verified for customer A:

* Joined on Jan 7th
* Purchased product 2 (curry) on Jan 7th
  → Appear in the result set as expected

---

### 🔗 Source

* RANK() & DENSE\_RANK(): [https://www.geeksforgeeks.org/rank-and-dense-rank-in-sql-server/](https://www.geeksforgeeks.org/rank-and-dense-rank-in-sql-server/)
* CTE: https://www.geeksforgeeks.org/cte-in-sql/
* DATEDIFF: https://www.w3schools.com/sql/func_sqlserver_datediff.asp
---

Would you like to challenge yourself with a follow-up: *What is the most common first item purchased across all members?* That would turn this into an insight for campaign design.


In [8]:
# Version 1 - Subquery (Less Readable)
query = """
SELECT
    df.customer_id,
    mn.product_name
FROM(
    SELECT
        *,
        DATEDIFF('day', m.join_date, s.order_date) AS DateDiff,
        RANK() OVER (PARTITION BY s.customer_id ORDER BY DATEDIFF('day', m.join_date, s.order_date) ASC) AS Rank
    FROM
        sales s
    INNER JOIN 
        members m
    ON s.customer_id = m.customer_id
    WHERE s.order_date >= m.join_date
) AS df

INNER JOIN 
    menu as mn
ON df.product_id = mn.product_id

WHERE Rank = 1;
"""

# Version - CTE (More readable)

query = """
WITH ranked_purchases AS (

    SELECT 
        s.customer_id,
        s.product_id,
        s.order_date,
        m.join_date,
         -- Rank purchases made after membership by earliest order_date
        RANK() OVER (
            PARTITION BY s.customer_id 
            ORDER BY s.order_date ASC
        ) AS purchase_rank -- 
    
    FROM sales s
    INNER JOIN
        members m
    ON s.customer_id = m.customer_id
    
    -- Only include purchases made on or after membership start
    WHERE s.order_date >= m.join_date
)

-- Join to menu to retrieve product name for the first purchase
-- Filter to get only the first-ranked item(s) after membership

SELECT
    rp.customer_id,
    m.product_name
FROM 
    ranked_purchases rp
INNER JOIN 
    menu m
ON rp.product_id = m.product_id
WHERE rp.purchase_rank = 1;

"""

result = con.execute(query).fetchdf()
result

,customer_id,product_name
0,B,sushi
1,A,curry
